In [1]:
import pandas as pd
df = pd.read_csv("speeches.csv.tar.gz")

/tmp/ipykernel_1955996/466859130.py:2: DtypeWarning: Columns (11,12,13,14,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("speeches.csv.tar.gz")


In [2]:
# Filtering
# Select only SI speeches:
c0 = df.country == "SI"

# Keep only MPs
c1 = (df.Speaker_MP == "MP")# & (df.Speaker_minister == "notMinister")

# Keep only speeches where speaker is either Opposition or Coalition:
c2 = df.Party_status.isin(["Opposition", "Coalition"])

# Keep only people that have at least 10 speeches:
gb = df[c0&c1&c2].groupby("Speaker_name").logits_pondered.count().reset_index()
speakers_to_keep = gb.Speaker_name[gb.logits_pondered >= 10]
c3 = df.Speaker_name.isin(speakers_to_keep)


ndf = df[c0&c1&c2&c3]
print(f"Available terms: \n{ndf.Term.value_counts()}")

Available terms: 
Term
4. mandat    55450
7. mandat    47127
5. mandat    46904
3. mandat    46136
8. mandat    44082
6. mandat    31315
Name: count, dtype: int64


In [3]:
# Selecting the term:
c0 = ndf.Term == "6. mandat"
nndf = ndf[c0].reset_index(drop=True)

In [4]:
gb1 = nndf.groupby(["Speaker_name", "Party_status"]).agg({
    "logits_pondered": ["mean", "count"]
}).reset_index()
gb1 = gb1.set_axis(gb1.columns.map(lambda l: '_'.join(l).rstrip("_").replace("logits_pondered_", "")), axis=1,)
gb2 = nndf.groupby(["Speaker_name"]).agg({
    "logits_pondered": ["mean", "count"]
}).reset_index()

gb2 = gb2.set_axis(gb2.columns.map(lambda l: '_'.join(l).rstrip("_").replace("logits_pondered_", "")), axis=1,)

gb2["Coalition_mean"] = gb2[["Speaker_name"]].merge(gb1[gb1["Party_status"]=="Coalition"], on="Speaker_name", how="left")["mean"]
gb2["Coalition_count"] = gb2[["Speaker_name"]].merge(gb1[gb1["Party_status"]=="Coalition"], on="Speaker_name", how="left")["count"]
gb2["Opposition_mean"] = gb2[["Speaker_name"]].merge(gb1[gb1["Party_status"]=="Opposition"], on="Speaker_name", how="left")["mean"]
gb2["Opposition_count"] = gb2[["Speaker_name"]].merge(gb1[gb1["Party_status"]=="Opposition"], on="Speaker_name", how="left")["count"]
gb2["Coal minus Oppo"] = gb2.Coalition_mean - gb2.Opposition_mean
gb2 = gb2.sort_values(by="mean", ascending=True)
gb2

,Speaker_name,mean,count,Coalition_mean,Coalition_count,Opposition_mean,Opposition_count,Coal minus Oppo
34,"Jerovšek, Jožef",1.262608,446,1.400194,127.0,1.207832,319.0,0.192363
27,"Jakič, Roman",1.393583,85,1.679099,4.0,1.379483,81.0,0.299616
92,"Černač, Zvonko",1.541843,252,4.061249,1.0,1.531805,251.0,2.529443
7,"Bratušek, Alenka",1.577959,144,NaN,NaN,1.577959,144.0,NaN
65,"Pojbič, Marijan",1.578197,147,1.830967,20.0,1.538391,127.0,0.292576
...,...,...,...,...,...,...,...,...
60,"Pepelnik, Truda",3.063929,27,3.063929,27.0,NaN,NaN,NaN
87,"Virant, Gregor",3.078745,2627,3.079409,2614.0,2.945337,13.0,0.134071
80,"Tomc, Romana",3.151318,3625,3.155055,1527.0,3.148599,2098.0,0.006456
10,"Brunskole, Renata",3.154896,4174,3.154555,2593.0,3.155456,1581.0,-0.000901
